## Imports + caminhos

In [1]:
# ===== Imports =====
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score
)

# ===== Caminhos =====
DATA_PATH = "../data/creditcard.csv"     # ajuste se precisar
IMG_DIR   = "../docs/Atividade1/img"     # ajuste se sua pasta for outra
os.makedirs(IMG_DIR, exist_ok=True)


## Etapa 1 — carregamento e limpeza

In [ ]:
df = pd.read_csv(DATA_PATH)
df.columns = [c.strip().lower() for c in df.columns]   # time, v1..v28, amount, class

# alvo e features
y = df["class"]                 # 0 = legítima, 1 = fraude
X = df.drop(columns=["class"])  # todo o resto são numéricos

print("shape:", df.shape)
print("fraudes (%):", (y==1).mean()*100)

## EDA

In [3]:
# distribuição do alvo
plt.figure()
y.value_counts().rename({0:"legit", 1:"fraud"}).plot(kind="bar")
plt.title("Distribuição do alvo (Class)")
plt.tight_layout()
plt.savefig(os.path.join(IMG_DIR, "dist_target.png"))
plt.close()

# histograma simples do Amount
plt.figure()
df["amount"].dropna().plot(kind="hist", bins=50)
plt.title("Histograma de Amount")
plt.tight_layout()
plt.savefig(os.path.join(IMG_DIR, "hist_amount.png"))
plt.close()

# estatísticas básicas úteis (opcional)
df[["time","amount"]].describe().to_csv(os.path.join(IMG_DIR, "basic_stats.csv"))

## Pré-processamento (binário 0/1) + Split

In [4]:
# Requisito: transformar tudo em 0/1 (one-hot de faixas)
# Estratégia simples: dividir cada coluna numérica em 5 faixas (quantis).
num_cols = X.columns.tolist()

preproc = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("impute", SimpleImputer(strategy="median")),              # valores ausentes -> mediana
            ("bin", KBinsDiscretizer(n_bins=5, encode="onehot-dense",  # 5 quantis -> colunas 0/1
                                     strategy="quantile"))
        ]), num_cols),
    ],
    remainder="drop"
)

# Split 70/30 estratificado
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)


## Treinamento + avaliação

In [5]:
# Árvore simples (poucos hiperparâmetros) — legível
clf = DecisionTreeClassifier(
    criterion="gini",
    class_weight="balanced",   # ajuda na classe rara (fraude)
    random_state=42,
    max_depth=5                # controla complexidade (simples!)
)

pipe = Pipeline([("prep", preproc), ("clf", clf)])
pipe.fit(X_train, y_train)

# Avaliação
y_pred = pipe.predict(X_test)
acc    = accuracy_score(y_test, y_pred)
print(f"Acurácia: {acc:.4f}")

# Relatório em arquivo (texto)
with open(os.path.join(IMG_DIR, "classification_report.txt"), "w", encoding="utf-8") as f:
    f.write(f"Accuracy: {acc:.4f}\n\n")
    f.write(classification_report(y_test, y_pred, digits=3))

# Matriz de confusão (imagem)
cm = confusion_matrix(y_test, y_pred, labels=pipe.named_steps["clf"].classes_)
ConfusionMatrixDisplay(cm, display_labels=pipe.named_steps["clf"].classes_).plot(xticks_rotation=45)
plt.title(f"Matriz de Confusão (Acc={acc:.3f})")
plt.tight_layout()
plt.savefig(os.path.join(IMG_DIR, "matriz_confusao.png"))
plt.close()


c:\Users\jp14h\OneDrive\Documentos\GitHub\repo-humberto\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


Acurácia: 0.9454


## Visual da árvore

In [6]:
# Pegar nomes das colunas após o one-hot (faixas)
preproc.fit(X_train)
feat_names = preproc.get_feature_names_out()

# Treinar uma árvore pequena só para visualizar os nós iniciais
from sklearn.tree import DecisionTreeClassifier, plot_tree
Xt = preproc.transform(X_train)
arvore = DecisionTreeClassifier(criterion="gini", class_weight="balanced", random_state=42, max_depth=3)
arvore.fit(Xt, y_train)

plt.figure(figsize=(20, 10))
plot_tree(arvore, max_depth=3, filled=True, class_names=["legit","fraud"], feature_names=feat_names)
plt.title("Árvore de Decisão (níveis iniciais) — features binárias")
plt.tight_layout()
plt.savefig(os.path.join(IMG_DIR, "arvore.png"))
plt.close()

print("Arquivos salvos em:", IMG_DIR)


c:\Users\jp14h\OneDrive\Documentos\GitHub\repo-humberto\env\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(


Arquivos salvos em: ../docs/Atividade1/img
